In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
data = pd.read_csv('./train_data_cleaning.csv')

In [3]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [4]:
tokenize('test what why how')

['test', 'what', 'why', 'how']

In [5]:
def stem(word):
    return stemmer.stem(word.lower())

In [6]:
stem('test')

'test'

In [7]:
def bag_of_words(tokenized_word,all_words):
    tokenized_word = [stem(w) for w in tokenized_word]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_word:
            bag[idx] = 1.0
    return bag

In [8]:
bag_of_words(['test','what !!!'],['test','what','what !!!'])

array([1., 0., 1.], dtype=float32)

In [9]:
X = data['text'].tolist()

In [10]:
y = data['target'].tolist()

In [11]:
all_words = []
new_data = []

In [12]:
tags = {}

In [13]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    new_data.append((new_X,y_batch))
    if y_batch not in list(tags.keys()):
        tags[y_batch] = 1
    else:
        tags[y_batch] += 1

7613it [00:02, 3160.50it/s]


In [14]:
tags

{1: 3271, 0: 4342}

In [15]:
tags = [0,1]

In [16]:
np.random.shuffle(all_words)
np.random.shuffle(new_data)

In [17]:
len(all_words)

132212

In [18]:
all_words = sorted(set(all_words))

In [19]:
len(all_words)

13928

In [20]:
X = []
y = []

In [21]:
for sentence,tag in tqdm(new_data):
    bag = bag_of_words(sentence,all_words)
    X.append(bag)
    label = tags.index(tag)
    y.append(label)

100%|██████████| 7613/7613 [00:22<00:00, 340.19it/s]


In [22]:
from sklearn.model_selection import *

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125)

In [25]:
device = 'cuda'

In [26]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()